# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [8]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from decimal import Decimal
from prettytable import PrettyTable


#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
## [0:'artist',
## 1:'firstName',
## 2:'gender',
## 3:'itemInSession',
## 4:'lastName',
## 5:'length',
## 6:'level',
## 7:'location',
## 8:'sessionId',
## 9:'song',
## 10:'userId']


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [2]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [3]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace and define querying functions

In [4]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

    
def execute_query(query):
    try:
        return session.execute(query)
    except Exception as e:
        print(e)
        
def execute_query_with_params(query, params):
    try:
        return session.execute(query, params)
    except Exception as e:
        print(e)
        
def process_csv_file(file, func):
    """
    Process all the data executing the given func for every *.json file of the given filepath
    :param filepath: The file path
    :return:None
    """
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader)
        for line in csvreader:
            func(line)
    
        

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### The table structure

As this query demands, we only need 2 identificators for this table: `session_id` and `item_in_session`. And finally only 3 more informations: `artist`, `song`, `song_length`.

With these specifications we have nicely distributed table accordingly with the `session_id` and clusterized by `item_in_session` which is a column with repeated values meaning that our clustering will generate sufficient clusters to have a great performance with this query.




In [9]:
execute_query("""
    CREATE TABLE IF NOT EXISTS session_library (
        session_id int,
        item_in_session int,
        artist text,
        song text,
        song_length decimal,
        PRIMARY KEY (session_id, item_in_session)
    )
""")

#### Data insertion

To insert the data we will do basic transforming in 3 fields:

- `session_id` should be an `int`
- `item_in_session` also should be an `int`
- `song_length` should be a `decimal`


In [10]:
def insert_to_session_library(line):
    query = "INSERT INTO session_library (session_id, item_in_session, artist, song, song_length)"
    query = query + " VALUES (%s, %s, %s, %s, %s)"
    queryParameters = (int(line[8]), int(line[3]), line[0], line[9], Decimal(line[5]))
    
    return execute_query_with_params(query, queryParameters)

process_csv_file('event_datafile_new.csv', func=insert_to_session_library)

#### Verifying the inserted data

Down below we have the stated query in the title of this section (and its results, of course):

```sql
SELECT 
    session_id, 
    item_in_session, 
    artist, 
    song, 
    song_length 
FROM 
    session_library 
WHERE 
    session_id = 338 AND 
    item_in_session  = 4
```

In [11]:
rows = execute_query("SELECT session_id, item_in_session, artist, song, song_length FROM session_library WHERE session_id = 338 AND item_in_session  = 4")
    
t = PrettyTable(['# Session', '# Session Item', 'Artist', 'Song', 'Song Duration'])
for row in rows:
    t.add_row([row.session_id, row.item_in_session, row.artist, row.song, row.song_length])
print(t)


+-----------+----------------+-----------+---------------------------------+---------------+
| # Session | # Session Item |   Artist  |               Song              | Song Duration |
+-----------+----------------+-----------+---------------------------------+---------------+
|    338    |       4        | Faithless | Music Matters (Mark Knight Dub) |    495.3073   |
+-----------+----------------+-----------+---------------------------------+---------------+


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### The table structure

As this query ask, we'll have two identificators: `user_id` and `session_id`. They should be the **partition keys** to avoid having different session items of the same user in different nodes of Cassandra.

We will have the following informations on our fields: `artist`, `song`, `first_name`, `last_name`.

Then we need to have the results ordered by the `item_in_session`. This is where the tricky part of this query lives. We need to add our `item_in_session` in the primary key to retrieve our items ordered by it.

In [12]:
execute_query("""
    CREATE TABLE IF NOT EXISTS user_library (
        user_id int, 
        session_id int,
        item_in_session int,
        artist text,
        song text,
        first_name text,
        last_name text,
        PRIMARY KEY ((user_id, session_id), item_in_session)
    )
""")

#### Data insertion

In this table, we will do only 3 transforms:

- `user_id` should be an `int`
- `session_id` should be an `int`
- `item_in_session` should be an `int`

In [13]:
def insert_to_user_library(line):
    query = "INSERT INTO user_library (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
    query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    queryParameters = (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4])
    
    return execute_query_with_params(query, queryParameters)

process_csv_file('event_datafile_new.csv', func=insert_to_user_library)


#### Testing query

Below we have a `SELECT` query translating the specifications of the title of this section:

```sql
SELECT 
    artist, 
    song, 
    first_name, 
    last_name, 
    item_in_session 
FROM 
    user_library 
WHERE 
    user_id = 10 AND 
    session_id = 182
```

In [14]:
rows = execute_query("SELECT artist, song, first_name, last_name, item_in_session FROM user_library WHERE user_id = 10 AND session_id = 182")

t = PrettyTable(['# Session Item', 'Artist', 'Song', 'First Name', 'Last Name'])
for row in rows:
    t.add_row([row.item_in_session, row.artist, row.song, row.first_name, row.last_name])
print(t)


+----------------+-------------------+------------------------------------------------------+------------+-----------+
| # Session Item |       Artist      |                         Song                         | First Name | Last Name |
+----------------+-------------------+------------------------------------------------------+------------+-----------+
|       0        |  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|       1        |    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
|       2        | Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|       3        |   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+----------------+-------------------+------------------------------------------------------+------------+-----------+


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### The table structure

As the title states, we have only one demand to our key, the `song` field. But only with the `song` field our rows would get overriden for different users that listened to the same song. Moreover, we need to add a second column to our `PRIMARY KEY` to make it **unique**, as we want the user's first and last name the best option to add to our `PRIMARY KEY` is the `user_id`, as it is the unique identificator of the user itself.

Then, we only need the `first_name`, `last_name` and the `artist` to complement the `song` column.

In [15]:
execute_query("""
    CREATE TABLE IF NOT EXISTS user_song_library (
        user_id int, 
        song text,
        artist text,
        first_name text,
        last_name text,
        PRIMARY KEY (song, user_id)
    )
""")       

#### Data insertion

To insert data into the `user_song_library` table, we will need to transform only the `user_id` to an `int`.

In [16]:
def insert_to_user_song_library(line):
    query = "INSERT INTO user_song_library (user_id, song, artist, first_name, last_name)"
    query = query + "VALUES (%s, %s, %s, %s, %s)"
    queryParameters = (int(line[10]), line[9], line[0], line[1], line[4])
    
    return execute_query_with_params(query, queryParameters)

process_csv_file('event_datafile_new.csv', func=insert_to_user_song_library)

#### Testing query

Finally, we will test the table with the query as this section title´s states:

```sql
SELECT 
    first_name, 
    last_name, 
    song, 
    artist 
FROM 
    user_song_library 
WHERE 
    song = 'All Hands Against His Own'
```

In [17]:
rows = execute_query("SELECT first_name, last_name, song, artist FROM user_song_library WHERE song = 'All Hands Against His Own'")

t = PrettyTable(['First Name', 'Last Name', 'Artist', 'Song'])
for row in rows:
    t.add_row([row.first_name, row.last_name, row.song, row.artist])
print(t)

+------------+-----------+---------------------------+----------------+
| First Name | Last Name |           Artist          |      Song      |
+------------+-----------+---------------------------+----------------+
| Jacqueline |   Lynch   | All Hands Against His Own | The Black Keys |
|   Tegan    |   Levine  | All Hands Against His Own | The Black Keys |
|    Sara    |  Johnson  | All Hands Against His Own | The Black Keys |
+------------+-----------+---------------------------+----------------+


### Drop the tables before closing out the sessions

In [18]:
execute_query("drop table user_library")
execute_query("drop table session_library")
execute_query("drop table user_song_library")


### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()